#Open Library Analysis - Big Data Computing Project
####Graph's Connected Components vs. k-means Clusters

In this notebook we analyse Open Library's data dumps freely downloadable from their website. https://openlibrary.org/developers/dumps
Books' data is probided in JSON records, which we preprocessed so as to keep only relevant information into a .csv file.

In the following we will clean the dataset and extract relevant features.

We use the resulting dataset to create a graph representing books affinity (i.e. nodes represent books and an edge connects two nodes only if their similarity is above a certain threshold). We are then interested in finding then connected components of such graph. 

Moreover, we will perform k-means clustering and compare, in terms of Silhouette Coefficient, the resulting clusters with the connected components of the aforementioned graph.

##Libraries

In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark import SparkContext, SparkConf

Dependencies, install the following libraries:

pip: `nltk`, `graphframes`

maven: `graphframes:graphframes:0.8.1-spark3.0-s_2.12`

In [0]:
spark.sparkContext.setCheckpointDir('/databricks/driver/checkpoints')

##Data Aquisition

Full dataset: 22.589.356 entries

In [0]:
%sh wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G" -O /tmp/books_long.csv.bz2 && rm -rf /tmp/cookies.txt

In [0]:
dbutils.fs.mv("file:/tmp/books_long.csv.bz2", "dbfs:/bdc-2020-21/datasets/books_long.csv.bz2")

##Parameters

In [0]:
DISTANCE_MEASURE = "cosine"
RANDOM_SEED = 42
# k = nCC , kmeans
MAX_ITERATIONS = 100 # maximum number of iterations of K-means EM algorithm
TOLERANCE = 0.000001 # tolerance between consecutive centroid updates (i.e., another stopping criterion)

SIM_THRESHOLD = 0.9 

##Functions

In [0]:
def load_dataframe(path):
  return spark.read.load(path, 
                         format="csv", 
                         sep=";", 
                         inferSchema="true", 
                         header="true"
                         )

In [0]:
def clean_dataframe(df):
  """This function takes the dataframe removing the features that have more than 70% of null valles, dropping any entries with null/duplicate key or title. 
  It also fill null values and substitute the key with an integer identifier.
  
  parameter: dataframe
  return: the cleaned dataframe
  """
  
  '''columns_to_drop = []
  df_size = df.count()
  for c in df.columns:
    if df.where(col(c).isNull()).count()/df_size > 0.7:
      columns_to_drop += [c]
  df = df.drop(*columns_to_drop)'''
  df = df.select("key", "title", "authors", "subjects")
  df = df.dropna(how="any", subset=["key", "title", "subjects", "authors"])
  df = df.dropDuplicates(['key'])
  df = df.dropDuplicates(['title'])
  #df = df.na.fill({'subjects': 'unknown', 'authors': 'unknown'})
  # This will return a new DF with all the columns + id
  df = df.select("key","title","authors","subjects", row_number().over(Window.partitionBy().orderBy(df['key'])).alias("id"))
  return df
    

In [0]:
# Tokenizer, StopWordsRemover, Word2Vec is for nlp
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer
# From Document_Clustering.ipynb
def clean_text(df, column_name, perform_stemming=True):
    """ 
    This function takes the raw text data and applies a standard NLP preprocessing pipeline consisting of the following steps:
      - Text cleaning
      - Tokenization
      - Stopwords removal
      - Stemming (Snowball stemmer)

    parameter: dataframe
    returns: the input dataframe along with the `cleaned_content` column as the results of the NLP preprocessing pipeline
    """
    # Text preprocessing pipeline
    # 1. Text cleaning
    # 1.a Case normalization
    lower_case_df = df.select(["id",lower(col(column_name)).alias(column_name)])
    # 1.b Trimming
    trimmed_df = lower_case_df.select(["id",trim(col(column_name)).alias(column_name)])
    # 1.c Filter out punctuation symbols
    no_punct_df = trimmed_df.select(["id",(regexp_replace(col(column_name), "[^a-zA-Z\\s]", "")).alias(column_name)])
    # 1.d Filter out any internal extra whitespace
    cleaned_df = no_punct_df.select(["id",trim(regexp_replace(col(column_name), " +", " ")).alias(column_name)])
    # 2. Tokenization (split text into tokens)
    tokenizer = Tokenizer(inputCol=column_name, outputCol="tokens")
    tokens_df = tokenizer.transform(cleaned_df).cache()
    # 3. Stopwords removal
    stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="terms")
    ret_df = stopwords_remover.transform(tokens_df).cache()
    # 4. Stemming (Snowball stemmer)
    if perform_stemming:
      stemmer = SnowballStemmer(language="english")
      stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
      ret_df = ret_df.withColumn("terms_stemmed", stemmer_udf("terms")).cache()
      
    return ret_df

In [0]:
from pyspark.ml.feature import Word2Vec 
#Word2Vec from Document_Clustering.ipynb 
def extract_w2v_features(df, column_name, out_col_name, EMBEDDING_SIZE = 20):
  word2vec = Word2Vec(vectorSize=EMBEDDING_SIZE, minCount=5, inputCol=column_name, outputCol=out_col_name, seed=RANDOM_SEED)
  model = word2vec.fit(df)
  features = model.transform(df).cache()
  
  return features

In [0]:
# VectorAssembler is for joining multiple vectors
from pyspark.ml.feature import VectorAssembler
def engineered_dataframe(df):
  # clean text
  clean_title_df = clean_text(df, "title")
  clean_subjects_df = clean_text(df, "subjects")
  clean_authors_df = clean_text(df, "authors", False)
  # extract vector embeddings
  w2v_title_features = extract_w2v_features(clean_title_df, "terms_stemmed", "title_vec").select(["id", "title_vec"])
  w2v_subjects_features = extract_w2v_features(clean_subjects_df, "terms_stemmed", "subjects_vec").select(["id", "subjects_vec"])
  w2v_authors_features = extract_w2v_features(clean_authors_df, "terms", "authors_vec", 10).select(["id", "authors_vec"])
  # join embedding dataframes
  df = w2v_title_features
  df = df.join(w2v_subjects_features, ["id"])
  df = df.join(w2v_authors_features, ["id"])
  # join features
  vec_ass = VectorAssembler(inputCols=["title_vec","subjects_vec","authors_vec"], outputCol="features", handleInvalid="keep")
  df = vec_ass.transform(df).select(["id", "features"])
  return df

In [0]:
def get_first_entries(df, n):
  return df.filter(col("id")<=n).cache()

In [0]:
from graphframes import *
sim_udf = udf(lambda x,y: float(x.dot(y))/float(x.norm(2)*y.norm(2)), DoubleType())
def build_graph(df, thr=SIM_THRESHOLD):
  nodes_df = df.select(["id"]).cache()
  sim_df = engineered_books_df .alias("src").join(engineered_books_df .alias("dst"), col("src.id") != col("dst.id"))\
    .select(
        col("src.ID").alias("src"), 
        col("dst.ID").alias("dst"), 
        sim_udf("src.features", "dst.features").alias("cos_sim"))\
    .sort(desc("cos_sim")).cache()
  edges_df = sim_df.filter(sim_df.cos_sim>thr).cache()
  return GraphFrame(nodes_df, edges_df)

In [0]:
def get_connected_components(g, df):
  CC_df = g.connectedComponents()
  n_CC = CC_df.select("component").dropDuplicates(["component"]).count()
  CC_df = CC_df.alias("comp").join(df.alias("orig").select("id", "features"), col("comp.id") == col("orig.id")).select("orig.id", "features", "component")
  return n_CC, CC_df

In [0]:
from pyspark.ml.clustering import KMeans
def k_means(dataset, 
            n_clusters, 
            distance_measure, 
            max_iter=MAX_ITERATIONS, 
            tol=TOLERANCE,
            features_col="features", 
            prediction_col="cluster", 
            random_seed=RANDOM_SEED):

  print("""Training K-means clustering using the following parameters: 
  - K (n. of clusters) = {:d}
  - max_iter (max n. of iterations) = {:d}
  - distance measure = {:s}
  - random seed = {:d}
  """.format(n_clusters, max_iter, distance_measure, random_seed))
  # Train a K-means model
  kmeans = KMeans(featuresCol=features_col, 
                   predictionCol=prediction_col, 
                   k=n_clusters, 
                   initMode="k-means||", 
                   initSteps=5, 
                   tol=tol, 
                   maxIter=max_iter, 
                   seed=random_seed, 
                   distanceMeasure=distance_measure)
  model = kmeans.fit(dataset)

  # Make clusters
  clusters_df = model.transform(dataset).cache()

  return model, clusters_df

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator
def evaluate_model(df, prediction_col):
  # Evaluate clusters by computing Silhouette score
  metric_name="silhouette"
  distance_measure=DISTANCE_MEASURE #"squaredEuclidean"
  evaluator = ClusteringEvaluator(metricName=metric_name,
                                distanceMeasure=distance_measure, 
                                predictionCol=prediction_col
                                )
  return evaluator.evaluate(df)

##Short df

In [0]:
# Read dataset file into a Spark Dataframe
books_df = load_dataframe("dbfs:/bdc-2020-21/datasets/books_long.csv.bz2")

In [0]:
cut = True
N_ENTRIES = 1000

In [0]:
if cut:
  engineered_books_df = get_first_entries(engineered_books_df, N_ENTRIES)

In [0]:
books_df = clean_dataframe(books_df)

In [0]:
engineered_books_df = engineered_dataframe(books_df)

cut again the dataset

###Graph

In [0]:
books_graph = build_graph(engineered_books_df)

In [0]:
K, components_df = get_connected_components(books_graph, engineered_books_df)
print(f'we have {K} connected components')

###Clustering

In [0]:
model, clusters_df = k_means(engineered_books_df, K, max_iter=MAX_ITERATIONS, distance_measure=DISTANCE_MEASURE)

###Evaluation

####Graph's connected components evaluation

In [0]:
score = evaluate_model(components_df, "component")

In [0]:
print(score)

####Clusters evaluation

In [0]:
score = evaluate_model(clusters_df, "cluster")

In [0]:
print(score)

## Long df

In [0]:
books_df = load_dataframe("dbfs:/bdc-2020-21/datasets/books_long.csv.bz2")

In [0]:
cleaned_books_df = clean_dataframe(books_df)

In [0]:
cleaned_books_df.count()

Out[21]: 17970902

In [0]:
engineered_books_df = engineered_dataframe(cleaned_books_df)

In [0]:
K_MIN = 2
K_MAX = 18
STEP = 2

In [0]:
def elbow_method(data, k_min=K_MIN, k_max=K_MAX, step=STEP, max_iter=MAX_ITERATIONS, distance_measure=DISTANCE_MEASURE):
  results = []
  for k in range(k_min, k_max, step):
    model, clusters_df = k_means(data, k, max_iter=max_iter, distance_measure=distance_measure)
    score = evaluate_model(clusters_df, "cluster")
    results.append([k, model.summary.trainingCost, score])

  return pd.DataFrame(results, columns = ['K', 'SSE', 'SCORE'])

In [0]:
elbow_results = elbow_method(w2v_features)

In [0]:
from matplotlib.ticker import MaxNLocator

def plot_elbow(results_df):
  fig, ax = plt.subplots(1,1, figsize =(8,6))
  _ = sns.lineplot(data=results_df, x="K", y="SSE", ax=ax)
  ax.set_xlabel('K (#clusters)')
  ax.set_ylabel('SSE')
  ax.xaxis.set_major_locator(MaxNLocator(integer=True))
  plt.show()

In [0]:
plot_elbow(elbow_results)

In [0]:

# StandardScaler is for PCA
from pyspark.ml.feature import StandardScaler, PCA
'''scaler = StandardScaler(inputCol="features", 
                        outputCol="std_vector_features",
                        withStd=True, withMean=True)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(engineered_books_df)

# Normalize each feature to have unit standard deviation.
engineered_books_df = scalerModel.transform(engineered_books_df)'''
'''n_PC = 10
pca_model = PCA(k=n_PC, inputCol="std_vector_features", outputCol="pca_features")
pca_features = pca_model.fit(engineered_books_df)
pca_books_df = pca_features.transform(engineered_books_df).cache()'''